# GAN met Fashion MNIST

In deze notebook gaan we opnieuw beelden genereren op basis van de Fashin MNIST dataset, net zoals in de vorige notebook over variational autoencoders (VAE). 
Hierbij moeten we een generator en discriminator opstellen.

## Importeren van packages en dataset

Eerst importeren we alle benodigde Python-bibliotheken voor het bouwen, trainen en visualiseren van onze VAE.
We gebruiken Pytorch voor het bouwen van het neurale netwerk, matplotlib voor visualisaties en NumPy voor numerieke berekeningen.
Daarna laden we de Fashion MNIST dataset, normaliseren de pixelwaarden naar de range [-1,1] 
en splitsen de dataset in een trainings- en testset. We gebruiken DataLoader om mini-batches te maken voor training.

In [ ]:
# Importeren van benodigde bibliotheken
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# Controleer of er een GPU beschikbaar is, zo niet gebruik de CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data-transformatie: normaliseer de afbeeldingen zodat de pixelwaarden tussen -1 en 1 liggen
transform = transforms.Compose([
    transforms.ToTensor(),  # Converteert beeld naar tensor
    transforms.Normalize((0.5,), (0.5,))  # Normaliseert naar bereik [-1, 1]
])

# FashionMNIST dataset downloaden en laden
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)

# DataLoader voor batches van de trainingsdata
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

print("Data geladen en DataLoader klaar.")

## Generator

 Deze cel definieert de generator die een random vector van ruis (latent vector) gebruikt om een afbeelding van 28x28 pixels te genereren. Het netwerk bestaat uit vier volledig verbonden lagen met ReLU-activatie, gevolgd door een Tanh activatie om de output te normaliseren.

## Discriminator

Deze cel definieert de discriminator, die een afbeelding van 28x28 pixels als invoer ontvangt en een enkele waarde teruggeeft die aangeeft of de afbeelding echt is (uit de dataset) of vals (gegenereerd door de generator). Het netwerk bestaat uit vier volledig verbonden lagen met LeakyReLU-activatie en dropout om overfitting te voorkomen.

## Loss functions

In deze cel worden de verliesfunctie en de optimalizers voor de generator en discriminator gedefinieerd. We gebruiken binaire cross-entropy (BCELoss) als verliesfunctie en de Adam-optimizer voor zowel de generator als de discriminator.

## Trainen van het GAN model

De discriminator wordt getraind met zowel echte als gegenereerde afbeeldingen om te leren onderscheid te maken tussen de twee, terwijl de generator wordt getraind om betere afbeeldingen te genereren die de discriminator niet kan onderscheiden van echte. Voor elke epoch printen we het verlies van zowel de generator als de discriminator.

## Visualiseren van de resultaten

Deze cel definieert een functie om enkele gegenereerde afbeeldingen van de getrainde generator te visualiseren. Het genereert willekeurige ruis en laat de generator nieuwe afbeeldingen maken, die vervolgens worden weergegeven met behulp van Matplotlib.